In [1]:
import os
import sys
import torchvision
from transformers.tokenization_bert import BertTokenizer

w_dir = %pwd
work_dir = os.path.dirname(w_dir)
work_dir

'/media/moju/data/work'

In [2]:
sys.path.append(w_dir+'/fgc_support_retri')

In [3]:
from train import train_model

In [4]:
train_model()

preprocessing /media/moju/data/work/fgc_support_retri/data/FGC/1.2/FGC_release_A_dev(cn).json ......


Expecting value: line 1 column 1 (char 0)
D108


  0%|          | 0/37 [00:00<?, ?it/s]

aspan error: 接触感染 
{'QTYPE': '进阶题', 'AMODE': 'Single-Span-Extraction', 'ASPAN': [{'text': '喉咙', 'end': 574, 'start': 572}, {'text': '粪便', 'end': 577, 'start': 575}, {'text': '传染力', 'end': 588, 'start': 585}, {'text': '接触感染', 'end': 800, 'start': 796}, {'text': '空气传播', 'end': 806, 'start': 802}, {'text': '两者皆有', 'end': 812, 'start': 808}], 'ATYPE': 'Misc', 'SHINT': [{'text': '\n疾管署表示，肠病毒感染者在发病前几天，喉咙与粪便就有病毒存在且具传染力，发病后一周内传染力最高，痊愈后肠病毒会随著粪便排出达8到12周之久', 'end': 622, 'start': 552}, {'text': ', Q:肠病毒的传染途径传染途径是接触感染还是空气传播或是两者皆有?, A:两者皆有', 'end': 821, 'start': 779}], 'QTEXT': '肠病毒的传染途径传染途径是接触感染还是空气传播或是两者皆有?', 'QID': 'D322Q05', 'ANSWER': [{'ATOKEN': [{'text': '两者皆有', 'start': -1}], 'ATEXT': '两者皆有'}]}
aspan error: 空气传播 
{'QTYPE': '进阶题', 'AMODE': 'Single-Span-Extraction', 'ASPAN': [{'text': '喉咙', 'end': 574, 'start': 572}, {'text': '粪便', 'end': 577, 'start': 575}, {'text': '传染力', 'end': 588, 'start': 585}, {'text': '接触感染', 'end': 800, 'start': 796}, {'text': '空气传播', 'end': 806, 'start': 802}, {'text'

Exception ignored in: <function _DataLoaderIter.__del__ at 0x7f9a7cdf6e60>
Traceback (most recent call last):
  File "/home/moju/anaconda3/envs/pytorch/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 675, in __del__
    def __del__(self):
KeyboardInterrupt
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/moju/anaconda3/envs/pytorch/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3326, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-4-4dc2ba0c028a>", line 1, in <module>
    train_model()
  File "/media/moju/data/work/fgc_support_retri/fgc_support_retri/train.py", line 66, in train_model
    labels=labels)
  File "/home/moju/anaconda3/envs/pytorch/lib/python3.7/site-packages/torch/nn/modules/module.py", line 493, in __call__
    result = self.forward(*input, **kwargs)
  File "/media/moju/data/work/fgc_support_retri/fgc_support_retri/sup_model.py", line 20, in forward
    sequence_output, pooled_output = self.bert_encoder(input_ids, token_type_ids, attention_mask)
  File "/home/moju/anaconda3/envs/pytorch/lib/python3.7/site-packages/torch/nn/modules/module.py", line 493, in __call__
    result = self.forward(*input, **kwargs)
  File "/home/moju/anaconda3/envs/pytorch/lib/python3.7

KeyboardInterrupt: 